In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_hi.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,292227,292227,292227,292227,292227,4424627005,4/3/20,951,PPP,FCH ENTERPRISES INC.,...,999 Other,7223 Special Food Services,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,NaN
1,292228,292228,292228,292228,292228,5613647004,4/6/20,951,PPP,NAVATEK LLC,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,NaN
2,292229,292229,292229,292229,292229,4281497103,4/13/20,951,PPP,"REINS INTERNATIONAL (USA) CO., LTD.",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,NaN
3,292230,292230,292230,292230,292230,5918468104,7/20/20,951,PPP,W2005 WKI REALTY LLC,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,NaN
4,292231,292231,292231,292231,292231,5683347210,4/27/20,951,PPP,"RESORTTRUST HAWAII, LLC",...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,297202,297202,297202,297202,297202,9153468404,2/16/21,951,PPS,TEMPTATION TOURS INC.,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,NaN
4976,297203,297203,297203,297203,297203,9206638401,2/16/21,951,PPS,AIWOHI BROS. INC.,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,NaN
4977,297204,297204,297204,297204,297204,9742598602,3/26/21,951,PPS,LELEIWI ELECTRIC INC.,...,999 Other,2382 Building Equipment Contractors,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,NaN
4978,297205,297205,297205,297205,297205,9937897210,4/28/20,951,PPP,"FALCON LANDSCAPING, LLC",...,999 Other,5617 Services to Buildings and Dwellings,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
4513,5148568309,"1630 Farrington Ave., Kulapuaa, HI",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,4424627005,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156
1,5613647004,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915
2,4281497103,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942
3,5918468104,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375
4,5683347210,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv")


In [ ]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv") 
fips_check_scraped.head() 

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [ ]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('15')] #HI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


  0%|          | 0/4979 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

4979


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1210]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_9923/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,4424627005,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,[150030025001006]
1,5613647004,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,[150030040001011]
2,4281497103,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,[150030020031001]
3,5918468104,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,[150030018012001]
4,5683347210,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,[150030005005001]
...,...,...,...,...,...
4975,9153468404,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,[150090319001108]
4976,9206638401,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,[150030086231001]
4977,9742598602,"99 Makaala St, Hilo, HI",19.702781,-155.067456,[150010206004015]
4978,9937897210,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,[150030075084001]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_9923/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,4424627005,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,150030025001006
1,5613647004,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,150030040001011
2,4281497103,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,150030020031001
3,5918468104,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,150030018012001
4,5683347210,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,150030005005001
...,...,...,...,...,...
4975,9153468404,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,150090319001108
4976,9206638401,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,150030086231001
4977,9742598602,"99 Makaala St, Hilo, HI",19.702781,-155.067456,150010206004015
4978,9937897210,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,150030075084001


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,4424627005,150030025001006
1,5613647004,150030040001011
2,4281497103,150030020031001
3,5918468104,150030018012001
4,5683347210,150030005005001
...,...,...
4975,9153468404,150090319001108
4976,9206638401,150030086231001
4977,9742598602,150010206004015
4978,9937897210,150030075084001


Read in base data

In [15]:
#if from geocode file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_hi.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
#fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,292227,292227,292227,292227,292227,292227,292227,292227,292227,292227,...,999 Other,7223 Special Food Services,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,NaN
1,292228,292228,292228,292228,292228,292228,292228,292228,292228,292228,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,NaN
2,292229,292229,292229,292229,292229,292229,292229,292229,292229,292229,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,NaN
3,292230,292230,292230,292230,292230,292230,292230,292230,292230,292230,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,NaN
4,292231,292231,292231,292231,292231,292231,292231,292231,292231,292231,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,297202,297202,297202,297202,297202,297202,297202,297202,297202,297202,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,NaN
4976,297203,297203,297203,297203,297203,297203,297203,297203,297203,297203,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,NaN
4977,297204,297204,297204,297204,297204,297204,297204,297204,297204,297204,...,999 Other,2382 Building Equipment Contractors,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,NaN
4978,297205,297205,297205,297205,297205,297205,297205,297205,297205,297205,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,NaN


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,292227,292227,292227,292227,292227,292227,292227,292227,292227,292227,...,7223 Special Food Services,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,NaN,150030025001006
1,292228,292228,292228,292228,292228,292228,292228,292228,292228,292228,...,NaN,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,NaN,150030040001011
2,292229,292229,292229,292229,292229,292229,292229,292229,292229,292229,...,7225 Restaurants and Other Eating Places,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,NaN,150030020031001
3,292230,292230,292230,292230,292230,292230,292230,292230,292230,292230,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,NaN,150030018012001
4,292231,292231,292231,292231,292231,292231,292231,292231,292231,292231,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,NaN,150030005005001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,297202,297202,297202,297202,297202,297202,297202,297202,297202,297202,...,5615 Travel Arrangement and Reservation Services,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,NaN,150090319001108
4976,297203,297203,297203,297203,297203,297203,297203,297203,297203,297203,...,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,NaN,150030086231001
4977,297204,297204,297204,297204,297204,297204,297204,297204,297204,297204,...,2382 Building Equipment Contractors,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,NaN,150010206004015
4978,297205,297205,297205,297205,297205,297205,297205,297205,297205,297205,...,5617 Services to Buildings and Dwellings,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,NaN,150030075084001


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,292227,292227,292227,292227,292227,292227,292227,292227,292227,292227,...,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,NaN,150030025001006,150030025001006
1,292228,292228,292228,292228,292228,292228,292228,292228,292228,292228,...,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,NaN,150030040001011,150030040001011
2,292229,292229,292229,292229,292229,292229,292229,292229,292229,292229,...,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,NaN,150030020031001,150030020031001
3,292230,292230,292230,292230,292230,292230,292230,292230,292230,292230,...,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,NaN,150030018012001,150030018012001
4,292231,292231,292231,292231,292231,292231,292231,292231,292231,292231,...,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,NaN,150030005005001,150030005005001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,297202,297202,297202,297202,297202,297202,297202,297202,297202,297202,...,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,NaN,150090319001108,150090319001108
4976,297203,297203,297203,297203,297203,297203,297203,297203,297203,297203,...,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,NaN,150030086231001,150030086231001
4977,297204,297204,297204,297204,297204,297204,297204,297204,297204,297204,...,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,NaN,150010206004015,150010206004015
4978,297205,297205,297205,297205,297205,297205,297205,297205,297205,297205,...,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,NaN,150030075084001,150030075084001


Drop Extra FIPS Columns

In [23]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,292227,292227,292227,292227,292227,292227,292227,292227,292227,292227,...,999 Other,7223 Special Food Services,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,150030025001006
1,292228,292228,292228,292228,292228,292228,292228,292228,292228,292228,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,150030040001011
2,292229,292229,292229,292229,292229,292229,292229,292229,292229,292229,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,150030020031001
3,292230,292230,292230,292230,292230,292230,292230,292230,292230,292230,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,150030018012001
4,292231,292231,292231,292231,292231,292231,292231,292231,292231,292231,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,150030005005001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,297202,297202,297202,297202,297202,297202,297202,297202,297202,297202,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,150090319001108
4976,297203,297203,297203,297203,297203,297203,297203,297203,297203,297203,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,150030086231001
4977,297204,297204,297204,297204,297204,297204,297204,297204,297204,297204,...,999 Other,2382 Building Equipment Contractors,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,150010206004015
4978,297205,297205,297205,297205,297205,297205,297205,297205,297205,297205,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,150030075084001


In [24]:
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
4513,296740,5148568309,1/25/21,951,PPS,COHI INC.,1630 Farrington Ave.,Kulapuaa,HI,96757.0,...,311 Food,3119 Other Food Manufacturing,3119.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1630 Farrington Ave., Kulapuaa, HI",NaN,NaN,NaN


Convert to File

In [25]:
fips_HI = fips_merge_drop


In [26]:
fips_HI.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv") 

Review & Compare

In [27]:
fips_HI = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/HI_fips_scraped.csv") 
fips_HI

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,292227,4424627005,4/3/20,951,PPP,FCH ENTERPRISES INC.,1765 SOUTH KING ST,HONOLULU,HI,...,999 Other,7223 Special Food Services,7223.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14
1,1,292228,5613647004,4/6/20,951,PPP,NAVATEK LLC,841 BISHOP ST ST 1110,HONOLULU,HI,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14
2,2,292229,4281497103,4/13/20,951,PPP,"REINS INTERNATIONAL (USA) CO., LTD.",307 Lewers St. #210 0.0,Honolulu,HI,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,HAWAII DISTRICT OFFICE,e $5-10 million,500 or more,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14
3,3,292230,5918468104,7/20/20,951,PPP,W2005 WKI REALTY LLC,2552 KALAKAUA AVENUE,HONOLULU,HI,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14
4,4,292231,5683347210,4/27/20,951,PPP,"RESORTTRUST HAWAII, LLC",5000 Kahala Ave,Honolulu,HI,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,e $5-10 million,250 to 499,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,4975,297202,9153468404,2/16/21,951,PPS,TEMPTATION TOURS INC.,15 S Wakea Ave Temptation Tours Inc,Kahului,HI,...,999 Other,5615 Travel Arrangement and Reservation Services,5615.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,1.500903e+14
4976,4976,297203,9206638401,2/16/21,951,PPS,AIWOHI BROS. INC.,91-1600 Wahane St Awiwohi Bros Inc,Kapolei,HI,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,1.500301e+14
4977,4977,297204,9742598602,3/26/21,951,PPS,LELEIWI ELECTRIC INC.,99 Makaala St,Hilo,HI,...,999 Other,2382 Building Equipment Contractors,2382.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"99 Makaala St, Hilo, HI",19.702781,-155.067456,1.500102e+14
4978,4978,297205,9937897210,4/28/20,951,PPP,"FALCON LANDSCAPING, LLC",1454 UILA ST.,HONOLULU,HI,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,1.500301e+14
